In [1]:
!kill -KILL 73666

/bin/bash: line 0: kill: (73666) - No such process


In [2]:
!nvidia-smi

Mon Jun  3 19:15:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:21:00.0 Off |                  Off |
| 30%   32C    P8              16W / 450W |      1MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from inference_chain import (
    get_inference_arguments,
    get_sampling_params,
    get_lora_request,
    format_data,
    initialize_engine,
    process_requests,
    filter_out_results,
    write_results
)

2024-06-03 19:15:28,609	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
#from unsloth import FastLanguageModel
#model, tokenizer = FastLanguageModel.from_pretrained(
#    model_name = "unsloth/gemma-2b-it",
#    max_seq_length = 1024,
#    dtype = None,
#    load_in_4bit = False
#)
#tokenizer.eos_token

In [5]:
import os
import json

from typing import List

from tqdm import tqdm
from vllm import SamplingParams, LLMEngine, EngineArgs
from vllm.lora.request import LoRARequest

from prompting import build_source_prompt, EOT_TOKEN, M1, MI, MA
from arguments import get_inference_arguments
from settings import OUTPUT_DIR

In [6]:
def run_chain_inference(m):

    data_dir = "/cluster/work/lawecon/Work/mbraasch/output/deepseek-7b-base-m1m2m3m4m58/data/test/normal/1.json"
    
    print(f"\n*** Start inference for model {m} ***")
    args.run_name = os.path.join(run_name, f'm{m[0]}-{m[-1]}' if isinstance(m, list) else f"m{m}")
    lora_request = get_lora_request(lora_path=args.run_name)
    print(f"Doing inference for model (at args.run_name) {args.run_name}.")

    instruction = M1 if m == 1 else (MA if isinstance(m, list) else MI)
    print(f"Using instruction: {instruction}")

    data = format_data(
        data_dir=data_dir,
        sampling_params=sampling_params,
        instruction=instruction,
        lora_request=lora_request,
        target_path=target_path
    )
    engine = initialize_engine(args) 
    results = process_requests(engine, data)
    results, finished = filter_out_results(results)
    write_results(args, results, finished, run_name)

In [7]:
def run_3_step_only_inference():
    data_dir = "/cluster/work/lawecon/Work/mbraasch/output/deepseek-7b-base-3-step-only/data/test_3_steps.json"
    print(f"\n*** Start inference for 3-step only model ***")
    args.run_name = os.path.join(OUTPUT_DIR, "deepseek-7b-base-3-step-only")
    lora_request = get_lora_request(lora_path=args.run_name)
    print(f"Doing inference for model {args.run_name}.")
    instruction = MA
    print(f"Using instruction: {instruction}")
    data = format_data(
        data_dir=data_dir,
        sampling_params=sampling_params,
        instruction=instruction,
        lora_request=lora_request
    )
    engine = initialize_engine(args)
    results = process_requests(engine, data)
    results, finished = filter_out_results(results)
    write_results(args, results, finished, run_name)

In [8]:
def run_baseline_inference(run_name):
    
    # Params
    delimiter = "\nFinal Answer: "
    sampling_params = get_sampling_params(eot_token="<eos>")
    data_dir = "/cluster/work/lawecon/Work/mbraasch/data/formatted/test.json"    
    instruction = "Solve the following math word problem step-by-step."#"Solve the following Math Word Problem"#MA
    
    # Run
    args = get_inference_arguments()
    args.run_name = os.path.join(OUTPUT_DIR, run_name)
    lora_request = get_lora_request(lora_path=args.run_name)
    data = format_data(
        data_dir=data_dir,
        sampling_params=sampling_params,
        instruction=instruction,
        lora_request=lora_request
    )
    engine = initialize_engine(args)
    results = process_requests(engine, data)
    results, finished = filter_out_results(results, delimiter=delimiter)
    write_results(args, results, finished, args.run_name)

In [9]:
run_name = "dl_my_format"
run_baseline_inference(run_name)

Initalizating sampling params.
Getting max adapter checkpoint from /cluster/work/lawecon/Work/mbraasch/output/dl_my_format.
Loaded ckpt /cluster/work/lawecon/Work/mbraasch/output/dl_my_format/checkpoint-372/adapter_model.
DP Load data from /cluster/work/lawecon/Work/mbraasch/data/formatted/test.json
Format data.
Initialize the LLMEngine.
INFO 06-03 19:15:30 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='google/gemma-2b-it', speculative_config=None, tokenizer='google/gemma-2b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=google/gemma-2b-it)


/cluster/home/mbraasch/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-03 19:15:31 utils.py:660] Found nccl from library /cluster/home/mbraasch/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-03 19:15:31 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 06-03 19:15:31 selector.py:32] Using XFormers backend.
WARNING 06-03 19:15:32 gemma.py:54] Gemma's activation function was incorrectly set to exact GeLU in the config JSON file when it was initially released. Changing the activation function to approximate GeLU (`gelu_pytorch_tanh`). If you want to use the legacy `gelu`, edit the config JSON to set `hidden_activation=gelu` instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
INFO 06-03 19:15:32 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 06-03 19:15:34 model_runner.py:175] Loading model weights took 4.6720 GB
INFO 06-03 19:15:35 gpu_executor.py:114] # GPU blocks: 51817, # CPU bloc

  0%|          | 0/1319 [00:00<?, ?it/s]

INFO 06-03 19:15:46 metrics.py:334] Avg prompt throughput: 5673.7 tokens/s, Avg generation throughput: 49.2 tokens/s, Running: 246 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 3.6%, CPU KV cache usage: 0.0%


...

INFO 06-03 19:16:51 metrics.py:334] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 1079.0 tokens/s, Running: 8 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 1.2%, CPU KV cache usage: 0.0%


100%|█████████▉| 1314/1319 [01:10<00:01,  2.64it/s]

INFO 06-03 19:16:56 metrics.py:334] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 574.4 tokens/s, Running: 5 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 1.1%, CPU KV cache usage: 0.0%


100%|██████████| 1319/1319 [01:19<00:00, 16.61it/s]

Filter out results.
Writing results to /cluster/work/lawecon/Work/mbraasch/output/dl_my_format/next_step_predictions.json.
Writing final results to /cluster/work/lawecon/Work/mbraasch/output/dl_my_format/final_results.json.


In [10]:
from calculate_metrics import calc_metrics

In [11]:
calc_metrics(
    ground_truth_path="/cluster/work/lawecon/Work/mbraasch/data/formatted/test.json",
    output_dir=f"/cluster/work/lawecon/Work/mbraasch/output/{run_name}"
)

Done.
